# EE 123 Lab 1 - Part 3: Acoustic Distance Measurement

### Building a Simple "Ultrasonic Ruler"

In this part, we'll turn our sonar system from Part 2 into a practical distance measurement tool. By measuring the time-of-flight of sound waves, we can calculate the distance to objects in front of our speaker/microphone setup.

## Setup

Run the cell below to import the necessary libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import sounddevice as sd

# Set default audio parameters
fs = 48000  # sampling rate
sd.default.samplerate = fs
sd.default.channels = 1

## Part I: Review - Key Functions from Part 2

Copy your `genChirpPulse()` and `crossCorr()` functions from Part 2 into the cell below.

In [ ]:
# Copy your genChirpPulse() function here
def genChirpPulse(N, f0, f1, fs):
    """
    Generate an analytic chirp pulse.
    
    Parameters:
        N: number of samples
        f0: start frequency (Hz)
        f1: end frequency (Hz)
        fs: sampling rate (Hz)
    Returns:
        Complex analytic chirp signal
    """
    # YOUR CODE HERE
    pass


# Copy your crossCorr() function here
def crossCorr(x, y):
    """
    Compute cross-correlation of x and y using FFT.
    
    Parameters:
        x: input signal
        y: reference signal
    Returns:
        Cross-correlation result
    """
    # YOUR CODE HERE
    pass

## Part II: The Speed of Sound

The speed of sound in air depends on temperature. A good approximation is:

$$v_{sound} = 331.3 + 0.606 \cdot T$$

where $T$ is the temperature in Celsius and $v_{sound}$ is in meters per second.

**Task:** Implement the function below to calculate the speed of sound.

In [ ]:
def speed_of_sound(temperature_celsius):
    """
    Calculate the speed of sound in air at a given temperature.
    
    Parameters:
        temperature_celsius: temperature in Celsius
    Returns:
        Speed of sound in m/s
    """
    # YOUR CODE HERE
    pass


# Test: at 20°C, speed should be about 343 m/s
print(f"Speed of sound at 20°C: {speed_of_sound(20):.1f} m/s")

## Part III: Time-of-Flight to Distance

When we send out a chirp and receive its echo, the sound travels to the object and back. Therefore:

$$d = \frac{v_{sound} \cdot \Delta t}{2}$$

where $\Delta t$ is the round-trip time (delay) and we divide by 2 because the sound travels the distance twice.

**Task:** Implement the function below to convert delay (in samples) to distance (in meters).

In [ ]:
def delay_to_distance(delay_samples, fs, temperature_celsius):
    """
    Convert delay in samples to distance in meters.
    
    Parameters:
        delay_samples: delay in number of samples
        fs: sampling rate (Hz)
        temperature_celsius: room temperature in Celsius
    Returns:
        Distance in meters
    """
    # YOUR CODE HERE
    # Hint: first convert samples to time, then use speed of sound
    pass

## Part IV: Building the Distance Meter

Now let's put it all together. The function below will:
1. Generate a chirp pulse
2. Play it and record the echo
3. Use cross-correlation to find the delay
4. Convert the delay to distance

**Task:** Complete the `measure_distance()` function.

In [ ]:
def measure_distance(temperature_celsius=20, f0=100, f1=10000, duration=0.1):
    """
    Measure distance to the nearest object using acoustic sonar.
    
    Parameters:
        temperature_celsius: room temperature
        f0: chirp start frequency (Hz)
        f1: chirp end frequency (Hz)
        duration: chirp duration (seconds)
    Returns:
        Measured distance in meters
    """
    # Number of samples for the chirp
    N = int(fs * duration)
    
    # Generate chirp pulse
    pulse = genChirpPulse(N, f0, f1, fs)
    
    # Create a signal with some silence after the chirp for recording echoes
    # We need enough time for the sound to travel and return
    max_distance = 5  # maximum measurable distance in meters
    max_delay_samples = int(2 * max_distance / speed_of_sound(temperature_celsius) * fs)
    
    tx_signal = np.zeros(N + max_delay_samples)
    tx_signal[:N] = pulse.real  # use real part for playback
    
    # Play and record
    recording = sd.playrec(tx_signal, fs, channels=1, blocking=True)
    recording = recording.flatten()
    
    # Cross-correlation to find the echo
    corr = crossCorr(recording, pulse)
    
    # Find the peak (skip the first peak which is the direct signal)
    # YOUR CODE HERE:
    # 1. Take the absolute value of corr
    # 2. Find the first major peak (this is the transmitted signal at delay=0)
    # 3. Find the second major peak (this is the echo from the object)
    # 4. The delay is the difference between these two peaks
    
    delay_samples = 0  # Replace with your calculation
    
    # Convert to distance
    distance = delay_to_distance(delay_samples, fs, temperature_celsius)
    
    return distance, corr

## Part V: Testing Your Distance Meter

**Experiment 1:** Place an object (book, wall, your hand) at a known distance from the speaker/microphone and measure it.

Tips:
- Start with larger distances (0.5 - 2 meters) which are easier to detect
- Use a hard, flat surface for better reflections
- Make sure the speaker and microphone point in the same direction

In [ ]:
# Set your room temperature
room_temp = 20  # Celsius

# Measure distance
distance, corr = measure_distance(temperature_celsius=room_temp)

print(f"Measured distance: {distance:.2f} m ({distance*100:.1f} cm)")

In [ ]:
# Plot the cross-correlation to visualize the echo detection
plt.figure(figsize=(12, 4))
time_ms = np.arange(len(corr)) / fs * 1000
plt.plot(time_ms, np.abs(corr))
plt.xlabel('Time (ms)')
plt.ylabel('Cross-correlation magnitude')
plt.title('Echo Detection')
plt.grid(True)
plt.show()